# Composite nodes

One of the convenient properties of Forney-style factor graphs (as opposed to regular factor graphs) is that they naturally allow for composability: one can draw a box around part of an FFG and treat this box as a new type of factor node. For example, one can combine a gain (multiplication) node with the addition node in a so-called *composite node*, as depicted in the following graph (Fig. 4.2 from Korl's [A factor graph approach to signal modelling, system identification and filtering](https://www.research-collection.ethz.ch/handle/20.500.11850/82737)):

<img src="./figures/information_filter.png" width="350"/>

Composite nodes are useful for two reasons:

1. Building large graphs becomes more convenient by 'packaging' repetitive parts of the graph as composite nodes.
2. One can define 'shortcut rules' for message updates, which might be more efficient and/or numerically stable than performing vanilla message passing on the internals of the composite node. For example, in the schedule shown above, message (4) is calculated directly from messages (2) and (3). The shortcut rule might exploit the matrix inversion lemma, or involve some optimization algorithm.

To demonstrate the use of composite nodes, in this demo we will build a gain-addition combination that constrains
\begin{align*}
    x_1 = x_0 + b\times u_1\,,
\end{align*}
where `x_0` and `u_1` have Gaussian priors, and b is a constant. We are interested in computing a belief over `x_1`.

We first construct a "flat" graph that represents the gain and addition contraints as two distinct factors, and generate a schedule for inferring a belief for `x_1`. Then, we compare the resulting schedule with a schedule generated on an FFG with a _composite_ gain-addition node. Finally, we show how to register a custom update rule with ForneyLab.

## Schedule generation without composite node

In [1]:
using ForneyLab

# Define factor graph for x1 = x0 + b*u1, where x0 and u1 have Gaussian priors, and b is a constant.
# This is a part of the information filter graph from the introduction.
g = FactorGraph()

b = [1.0; 0.5]

@RV x_0 ~ GaussianMeanVariance(ones(2), eye(2))
@RV u_1 ~ GaussianMeanVariance(1.0, 1.0)
@RV x_1 = x_0 + b*u_1;

In [2]:
flat_schedule = sumProductSchedule(x_1)

draw(g, schedule=flat_schedule) # Inspect the resulting schedule

println(flat_schedule)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.38.0 (20140413.2041)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 15164856479281966878 -->
 15164856479281966878 
 
 + 
 addition_1 
 
<!-- 13245520558991760283 -->
 13245520558991760283 
 
 × 
 multiplication_1 
 
<!-- 15164856479281966878--13245520558991760283 -->
 15164856479281966878--13245520558991760283 
 
 variable_1 
 1 out 
 (3) 
 3 in2 
 
<!-- 4987538676912453407 -->
 4987538676912453407 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 15164856479281966878--4987538676912453407 -->
 15164856479281966878--4987538676912453407 
 
 x_0 
 1 out 
 (1) 
 2 in1 
 
<!-- 6139520642863064252 -->
 6139520642863064252 
 
 clamp_4 
 
<!-- 2855161866637452889 -->
 2855161866637452889 
 
 clamp_5 
 
<!-- 13245520558991760283--2855161866637452889 -->
 13245520558991760283--2855161866637452889 
 
 clamp_5 
 1 out 
 3 a 
 
<!-- 5065954216277500908 -->
 5065954216277500908 
 
 𝒩 
 gaussianmeanvariance_2 
 
<!-- 13245520558991760283--5065954216277500908 -->
 13245520558991760283--5065954216277500908 
 
 u_1 
 1 out 
 (2) 
 2 in1 
 
<!-- 16009949549921374169 -->
 16009949549921374169 
 
 clamp_3 
 
<!-- 9968858608990250406 -->
 9968858608990250406 
 
 clamp_2 
 
<!-- 4987538676912453407--9968858608990250406 -->
 4987538676912453407--9968858608990250406 
 
 clamp_2 
 1 out 
 3 v 
 
<!-- 5867612639134477689 -->
 5867612639134477689 
 
 clamp_1 
 
<!-- 4987538676912453407--5867612639134477689 -->
 4987538676912453407--5867612639134477689 
 
 clamp_1 
 1 out 
 2 m 
 
<!-- 5065954216277500908--6139520642863064252 -->
 5065954216277500908--6139520642863064252 
 
 clamp_4 
 1 out 
 3 v 
 
<!-- 5065954216277500908--16009949549921374169 -->
 5065954216277500908--16009949549921374169 
 
 clamp_3 
 1 out 
 2 m 
 
<!-- 177889762523338687412 -->
 177889762523338687412 
 
<!-- 177889762523338687412--15164856479281966878 -->
 177889762523338687412--15164856479281966878 
 
 x_1 
 1 out 
 (4)

	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} clamp_1
	SPClamp{MatrixVariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.MatrixVariate} clamp_2
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance gaussianmeanvariance_1
	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} clamp_5
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_3
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_4
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance gaussianmeanvariance_2
3.	SPMultiplicationOutVGP on Interface 1 (out) of ForneyLab.Multiplication multiplication_1
4.	SPAdditionOutVGG on Interface 1 (out) of ForneyLab.Addition addition_1



## Usage of composite nodes in an FFG

Now we 'draw a box' around the multiplication and addition nodes, and create a composite node. We can easily define a composite node using ForneyLab's `@composite` macro.

In [3]:
# Define a composite node for z = x + b*y
@composite GainAddition (z, x, y) begin
    # Specify the 'internal factor graph' of the GainAddion composite node.
    # z, x, and y can be used as if they are existing Variables in this block.
    b = [1.0; 0.5]
    
    @RV z = x + b*y
end

Here, `GainAddition` is the name of the composite node that we're defining. The tuple `(z, x, y)` defines the variables that this node constrains. The order of these variables simultaneously fixes the argument order for the update rules. Now that our custom `GainAddition` composite node is defined, we can use it like any other factor node.

In [4]:
g2 = FactorGraph()

@RV x_0 ~ GaussianMeanVariance(ones(2), eye(2))
@RV u_1 ~ GaussianMeanVariance(1.0, 1.0)
@RV x_1 ~ GainAddition(x_0, u_1);

In [5]:
composite_schedule = sumProductSchedule(x_1)

draw(g2, schedule=composite_schedule)

println(composite_schedule)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.38.0 (20140413.2041)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 16571158628914658659 -->
 16571158628914658659 
 
 clamp_4 
 
<!-- 9870402200367274880 -->
 9870402200367274880 
 
 clamp_1 
 
<!-- 509946603488025155 -->
 509946603488025155 
 
 clamp_2 
 
<!-- 11757897953945898430 -->
 11757897953945898430 
 
 GainAddition 
 gainaddition_1 
 
<!-- 497303534006292767 -->
 497303534006292767 
 
 𝒩 
 gaussianmeanvariance_2 
 
<!-- 11757897953945898430--497303534006292767 -->
 11757897953945898430--497303534006292767 
 
 u_1 
 1 out 
 (2) 
 3 y 
 
<!-- 3579482220077376545 -->
 3579482220077376545 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 11757897953945898430--3579482220077376545 -->
 11757897953945898430--3579482220077376545 
 
 x_0 
 1 out 
 (1) 
 2 x 
 
<!-- 497303534006292767--16571158628914658659 -->
 497303534006292767--16571158628914658659 
 
 clamp_4 
 1 out 
 3 v 
 
<!-- 9601466200163410288 -->
 9601466200163410288 
 
 clamp_3 
 
<!-- 497303534006292767--9601466200163410288 -->
 497303534006292767--9601466200163410288 
 
 clamp_3 
 1 out 
 2 m 
 
<!-- 3579482220077376545--9870402200367274880 -->
 3579482220077376545--9870402200367274880 
 
 clamp_1 
 1 out 
 2 m 
 
<!-- 3579482220077376545--509946603488025155 -->
 3579482220077376545--509946603488025155 
 
 clamp_2 
 1 out 
 3 v 
 
<!-- 38629067168912398892 -->
 38629067168912398892 
 
<!-- 38629067168912398892--11757897953945898430 -->
 38629067168912398892--11757897953945898430 
 
 x_1 
 1 z 
 (3)

	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} clamp_1
	SPClamp{MatrixVariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.MatrixVariate} clamp_2
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance gaussianmeanvariance_1
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_3
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_4
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance gaussianmeanvariance_2
3.	(INTERNAL SCHEDULE) SPAdditionOutVGG on Interface 1 (z) of GainAddition gainaddition_1



Here, the resulting schedule contains one less message than before, because message (3) directly computes the belief over `x_1` from the prior beliefs. Currently, we have not defined a composite update rule yet and, as a default, message (3) is computed by internal message passing in the composite node. We can inspect this internal schedule.

In [6]:
println(composite_schedule[end].internal_schedule)

	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} clamp_1
1.	SPMultiplicationOutVGP on Interface 1 (out) of ForneyLab.Multiplication multiplication_1
2.	SPAdditionOutVGG on Interface 1 (out) of ForneyLab.Addition addition_1



## Defining custom shortcut rules

If we actually want to use composite nodes to get a different (i.e. more efficient) algorithm, we'll have to specify message update rules that apply specifically to the composite nodes at hand. These rules are sometimes referred to as _shortcut rules_, since they provide a way to shortcut the calculation of internal messages. This rule definition for the composite nodes is analogous to the rule definitions for regular nodes. If we then build a new sum-product algorithm, the custom shortcut rule `SPGainAdditionOutVGG` will be automatically inserted.

In [7]:
@sumProductRule(:node_type     => GainAddition,                                 # our custom composite node
                :outbound_type => Message{GaussianMeanPrecision},               # this rule produces a GaussianMeanPrecision msg
                :inbound_types => (Void, Message{Gaussian}, Message{Gaussian}), # msg towards first interface, incoming types
                :name          => SPGainAdditionOutVGG)                         # name of the update rule;

In [8]:
shortcut_schedule = sumProductSchedule(x_1)

println(shortcut_schedule)

	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} clamp_1
	SPClamp{MatrixVariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.MatrixVariate} clamp_2
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance gaussianmeanvariance_1
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_3
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_4
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance gaussianmeanvariance_2
3.	SPGainAdditionOutVGG on Interface 1 (z) of GainAddition gainaddition_1

